## Model Notebook

In [1]:
# intents
# 1. Greetings
# 2. enrollment
# 3. pricing
# 4. Track
# 5. Fallback_input

In [237]:
import pandas as pd

In [291]:
df = pd.read_csv('training_phrase.csv')[['phrase','intent']]

In [292]:
# Convert the dictionary to a DataFrame
#df = pd.DataFrame([(phrase,intent) for intent, phrases in training_phrases.items() for phrase in phrases], columns=["phrase","intent"])

In [295]:
df['phrase'] = df['phrase'].str.lower()

In [296]:
df.intent.value_counts()

intent
greetings     150
enrollment    129
fallback      110
pricing       101
tracking       90
Name: count, dtype: int64

In [297]:
df.dropna(inplace=True)

In [298]:
#df.to_csv('training_phrase.csv')

In [299]:
#df = pd.concat([df, df_fallback])
df.head()

,phrase,intent
0,hello,greetings
1,hi,greetings
2,hey,greetings
3,good day,greetings
4,howdy,greetings


In [300]:
df.intent.unique()

array(['greetings', 'enrollment', 'pricing', 'tracking', 'fallback'],
      dtype=object)

In [301]:
df.shape

(580, 2)

In [302]:
target_map = {'greetings':0, 'enrollment':1, 'pricing':2, 'tracking':3, 'fallback':4}

In [303]:
df['target'] = df['intent'].map(target_map)

In [304]:
df

,phrase,intent,target
0,hello,greetings,0
1,hi,greetings,0
2,hey,greetings,0
3,good day,greetings,0
4,howdy,greetings,0
...,...,...,...
576,too many words,fallback,4
577,hmmmmmm,fallback,4
578,speak in french,fallback,4
579,i don’t want to talk,fallback,4


In [305]:
df['target'].value_counts()

target
0    150
1    129
4    110
2    101
3     90
Name: count, dtype: int64

In [306]:
df.target.unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [307]:
df.intent.unique()

array(['greetings', 'enrollment', 'pricing', 'tracking', 'fallback'],
      dtype=object)

In [308]:
#import spacy
#nlp = spacy.load("en_core_web_lg")

In [309]:
#use this utility function to preprocess the text
#1. Remove the stop words
#2. Convert to base form using lemmatisation

#def preprocess(text):
#    doc = nlp(text)
#    filtered_tokens = []
#    for token in doc:
#        if token.is_stop or token.is_punct:
#            continue
#        filtered_tokens.append(token.lemma_)
#    return ' '.join(filtered_tokens)

#create a new column "preprocessed_text" which store the clean form of given text [use apply and lambda function]
#df['preprocessed_text'] = df['phrase'].apply(preprocess)


In [310]:
df

,phrase,intent,target
0,hello,greetings,0
1,hi,greetings,0
2,hey,greetings,0
3,good day,greetings,0
4,howdy,greetings,0
...,...,...,...
576,too many words,fallback,4
577,hmmmmmm,fallback,4
578,speak in french,fallback,4
579,i don’t want to talk,fallback,4


### spacy word embedding

In [311]:
#create a new column "vector" that store the vector representation of each pre-processed text
df['vector'] = df['phrase'].apply(lambda x: nlp(x).vector)

In [312]:
### splitting into train and test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.vector,
    df.target,
    test_size=0.2,
    random_state=2022,
    stratify=df.target
)

In [313]:
from sklearn.metrics import classification_report

In [314]:
import numpy as np

#reshapes the X_train and X_test using 'stack' function of numpy. Store the result in new variables "X_train_2d" and "X_test_2d"
X_train_2d = np.stack(X_train)
X_test_2d =  np.stack(X_test)

In [315]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
scaled_train_embed = scaler.fit_transform(X_train_2d)
scaled_test_embed = scaler.transform(X_test_2d)

In [316]:
from sklearn.svm import SVC

clf = SVC()

#2. fit with all_train_embeddings and y_train
clf.fit(X_train_2d, y_train)

#3. get the predictions for all_test_embeddings and store it in y_pred
y_pred = clf.predict(X_test_2d)

#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.73      0.83        30
           1       0.84      1.00      0.91        26
           2       1.00      0.90      0.95        20
           3       1.00      1.00      1.00        18
           4       0.73      0.86      0.79        22

    accuracy                           0.89       116
   macro avg       0.91      0.90      0.90       116
weighted avg       0.90      0.89      0.89       116



### SVM gave the best performance even without preprocessing

In [110]:
text = 'i really want to register for '

In [114]:
text = 'hello common nothing is here'

In [317]:
text = 'hi how have you been'

In [318]:
vector = nlp(text).vector

In [319]:
text_vector =  np.stack(vector)

In [320]:
clf.predict([text_vector])[0]

0

In [321]:
import pickle

In [322]:
# Save the trained pipeline
with open("intent_classifier.pkl", "wb") as f:
    pickle.dump(clf, f)

In [323]:
# Load the saved pipeline
with open("intent_classifier.pkl", "rb") as f:
    loaded_pipeline = pickle.load(f)

# Example usage
test_phrase = "I'm bored"
vector = nlp(test_phrase).vector
text_vector =  np.stack(vector)
predicted_intent = loaded_pipeline.predict([text_vector])
print(predicted_intent[0])


4


In [324]:
target_response = {0:'greetings', 1:'enrollment',2:'pricing', 3:'tracking',4:'fallback'}

In [325]:
response = {
    'greetings':['Nice, you are very much welcome to poshem, what do you want to do ? Register for a course ? \nCheck the price of our courses? \nConfirm your registered courses?'],
    'enrollment':'kindly request for the course you want to register in this form\n(e.g) My name is Promise, i am 30 years old, B-Tech computer science, this is my phone number: 07063083925, i want to learn Excel, this is my Phone number: 07063083925'
}

In [220]:
def user_input_to_model(user_input):
    vector = nlp(user_input).vector
    text_vector =  np.stack(vector)
    predicted_intent = loaded_pipeline.predict([text_vector])
    return predicted_intent
    
def response_function(user_intent):
    if user_intent == 0:
        return response['greetings'][0]
    elif user_intent == 1:
        return response['enrollment']
    elif user_intent == 2:
        return target_response[2]
    elif user_intent == 3:
        return target_response[3]
    elif user_intent == 4:
        return target_response[4]
    else:
        return user_intent

In [223]:
welcome_message = "Hello! Welcome to Poshem Technologies, a fair greeting will be okay for us to attend to you: "
print(welcome_message)

while True:
    user_input = input()
    user_intent = user_input_to_model(user_input)
    bot_response = response_function(user_intent)
    
    if user_input.lower() == "i am done":  # or any other condition
        print('Thanks for Choosing Poshem Technologies Institution')
        break
    else:
        print(bot_response)

Hello! Welcome to Poshem Technologies, a fair greeting will be okay for us to attend to you: 


 hi


Nice, you are very much welcome to poshem, what do you want to do ? Register for a course ? 
Check the price of our courses? 
Confirm your registered courses?


 i am done


Thanks for Choosing Poshem Technologies Institution


In [199]:
print('Nice, you are very much welcome to poshem, what do you want to do ? Register for a course? \nCheck the price of our courses? \nConfirm your registered courses?')  

Nice, you are very much welcome to poshem, what do you want to do ? Register for a course? 
Check the price of our courses? 
Confirm your registered courses?


In [288]:
import numpy as np

target_response = {
    0: 'greetings',
    1: 'enrollment',
    2: 'pricing',
    3: 'tracking',
    4: 'fallback'
}

fallback_responses = [
    "Sorry, I didn’t quite understand that. Can you rephrase, read my previous prompt and respond accordingly?",
    "Hmm, I'm not sure what you mean. Try asking about enrollment or pricing, read my previous prompt and respond accordingly.",
    "Let’s try that again. Ask about courses, prices, or tracking progress, read my previous prompt and respond accordingly"
]

# You can customize this based on your chatbot personality
response = {
    'greetings':['Nice, you are very much welcome to poshem, what do you want to do ? Register for a course ? \nCheck the price of our courses? \nConfirm your registered courses?'],
    'enrollment':'kindly request for the course you want to register in this form\n(e.g) My name is Promise, i am 30 years old, B-Tech computer science, this is my phone number: 07063083925, i want to learn Excel, this is my Phone number: 07063083925',
    'pricing': "Our pricing varies by course. Here is the breakdown?",
    'tracking': "To track your course, Kindly let me know your ID.",
    'fallback': fallback_responses
}

def user_input_to_model(user_input):
    vector = nlp(user_input).vector
    predicted_intent = loaded_pipeline.predict([vector])[0]  # Extract the single prediction
    return predicted_intent

def response_function(user_intent):
    if user_intent == 0:
        return response['greetings'][0]
    elif user_intent == 1:
        return response['enrollment']
    elif user_intent == 2:
        return response['pricing']
    elif user_intent == 3:
        return response['tracking']
    elif user_intent == 4:
        # Randomly pick a fallback message to make responses feel dynamic
        return np.random.choice(response['fallback'])
    else:
        return "Oops, something went wrong."

# Start chatbot loop
welcome_message = "Bot: Hello! Welcome to Poshem Technologies, a fair greeting will be okay for us to attend to you: "
print(welcome_message)

while True:
    user_input = input("You: ")
    
    if user_input.lower() in ["i am done", "exit", "quit"]:
        print('Bot: Thanks for Choosing Poshem Technologies Institution. This is your order Id')
        break

    user_intent = user_input_to_model(user_input)
    bot_response = response_function(user_intent)
    print(f"Bot: {bot_response}")


Bot: Hello! Welcome to Poshem Technologies, a fair greeting will be okay for us to attend to you: 


You:  I am bored


Bot: Hmm, I'm not sure what you mean. Try asking about enrollment or pricing, read my previous prompt and respond accordingly.


You:  Hi


Bot: Nice, you are very much welcome to poshem, what do you want to do ? Register for a course ? 
Check the price of our courses? 
Confirm your registered courses?


...

Bot: kindly request for the course you want to register in this form
(e.g) My name is Promise, i am 30 years old, B-Tech computer science, this is my phone number: 07063083925, i want to learn Excel, this is my Phone number: 07063083925


You:  kindly show me the courses i registered


Bot: To track your course, Kindly let me know your ID.


You:  exit


Bot: Thanks for Choosing Poshem Technologies Institution. This is your order Id


In [327]:
# Intent mapping
target_response = {
    0: 'greetings',
    1: 'enrollment',
    2: 'pricing',
    3: 'tracking',
    4: 'fallback'
}

# Response templates
fallback_responses = [
    "Sorry, I didn’t quite understand that. Can you rephrase, read my previous prompt and respond accordingly?",
    "Hmm, I'm not sure what you mean. Try asking about enrollment or pricing, read my previous prompt and respond accordingly.",
    "Let’s try that again. Ask about courses, prices, or tracking progress, read my previous prompt and respond accordingly"
]

response = {
    'greetings': [
        "Nice, you are very much welcome to Poshem! What do you want to do?\n"
        "- Register for a course?\n- Check the price of our courses?\n- Confirm your registered courses?"
    ],
    'enrollment': (
        "Kindly request for the course you want to register in this format:\n"
        "(e.g.) My name is Promise, I am 30 years old, B-Tech Computer Science.\n"
        "This is my phone number: 07063083925.\n"
        "I want to learn Excel."
    ),
    'pricing': "Our pricing varies by course. Here is the breakdown...",
    'tracking': "To track your course, kindly let me know your ID.",
    'fallback': fallback_responses
}

# Convert user input to predicted intent
def user_input_to_model(user_input):
    vector = nlp(user_input).vector
    predicted_intent = loaded_pipeline.predict([vector])[0]  # Extract single prediction
    return predicted_intent

# Generate bot response based on predicted intent
def response_function(user_intent):
    intent_label = target_response.get(user_intent, 'fallback')

    if intent_label == 'fallback':
        return np.random.choice(response['fallback'])

    reply = response.get(intent_label)
    return reply[0] if isinstance(reply, list) else reply

# Start the chatbot loop
print("Bot: Hello! Welcome to Poshem Technologies, a fair greeting will be okay for us to attend to you:")

while True:
    user_input = input("You: ").strip()
    user_input = user_input.lower()

    if user_input.lower() in ["i am done", "exit", "quit"]:
        print('Bot: Thanks for choosing Poshem Technologies Institution. This is your order ID.')
        break

    try:
        user_intent = user_input_to_model(user_input)
        intent_label = target_response.get(user_intent, 'fallback')

        bot_response = response_function(user_intent)
        print(f"Bot: {bot_response}")

    except Exception as e:
        print("Bot: Sorry, I couldn't process that. Please try again or ask for help.")


Bot: Hello! Welcome to Poshem Technologies, a fair greeting will be okay for us to attend to you:


You:  Hi


Bot: Nice, you are very much welcome to Poshem! What do you want to do?
- Register for a course?
- Check the price of our courses?
- Confirm your registered courses?


KeyboardInterrupt: Interrupted by user

### improvement

In [8]:
import pickle
import spacy
import numpy as np
import logging
from datetime import datetime
import json

# Load the saved pipeline
with open("intent_classifier.pkl", "rb") as f:
    loaded_pipeline = pickle.load(f)
nlp = spacy.load("en_core_web_lg")

# Set up logging to file
logging.basicConfig(
    filename='chat_log.json',
    level=logging.INFO,
    format='%(message)s'
)

# Intent mapping
target_response = {
    0: 'greetings',
    1: 'enrollment',
    2: 'pricing',
    3: 'tracking',
    4: 'fallback'
}

# Response templates
fallback_responses = [
    "Sorry, I didn’t quite understand that. Can you rephrase, read my previous prompt and respond accordingly?",
    "Hmm, I'm not sure what you mean. Try asking about enrollment or pricing, read my previous prompt and respond accordingly.",
    "Let’s try that again. Ask about courses, prices, or tracking progress, read my previous prompt and respond accordingly"
]

response = {
    'greetings': [
        "Nice, you are very much welcome to Poshem! What do you want to do?\n"
        "- Register for a course?\n- Check the price of our courses?\n- Confirm your registered courses?"
    ],
    'enrollment': (
        "Kindly request for the course you want to register in this format:\n"
        "(e.g.) My name is Promise, I am 30 years old, B-Tech Computer Science.\n"
        "This is my phone number: 07063083925.\n"
        "I want to learn Excel."
    ),
    'pricing': "Our pricing varies by course. Here is the breakdown...",
    'tracking': "To track your course, kindly let me know your ID.",
    'fallback': fallback_responses
}

# Function to simulate intent prediction
def user_input_to_model(user_input):
    vector = nlp(user_input).vector
    predicted_intent = loaded_pipeline.predict([vector])[0]  # Extract single prediction
    return predicted_intent

# Function to generate bot response
def response_function(user_intent):
    intent_label = target_response.get(user_intent, 'fallback')

    if intent_label == 'fallback':
        return np.random.choice(response['fallback'])

    reply = response.get(intent_label)
    return reply[0] if isinstance(reply, list) else reply

# Chat history logging function
def log_interaction(user_input, predicted_intent, bot_response):
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "user_input": user_input,
        "predicted_intent": target_response.get(predicted_intent, 'fallback'),
        "bot_response": bot_response
    }
    logging.info(json.dumps(log_entry))

# Start chatbot loop
print("Bot 🤖: Hello! Welcome to Poshem Technologies, a fair greeting will be okay for us to attend to you:")

while True:
    user_input = input("You 🫥: ").strip()

    if user_input.lower() in ["i am done", "exit", "quit"]:
        print("Bot 🤖: Thanks for choosing Poshem Technologies Institution. This is your course ID. You'll get a mail from us shortly")
        break

    try:
        predicted_intent = user_input_to_model(user_input)
        bot_response = response_function(predicted_intent)
        print(f"Bot 🤖: {bot_response}")

        # Log the conversation
        log_interaction(user_input, predicted_intent, bot_response)

    except Exception as e:
        print("Bot 🤖: Sorry, I couldn't process that. Please try again or ask for help.")


Bot 🤖: Hello! Welcome to Poshem Technologies, a fair greeting will be okay for us to attend to you:


You 🫥:  Greetings


Bot 🤖: Nice, you are very much welcome to Poshem! What do you want to do?
- Register for a course?
- Check the price of our courses?
- Confirm your registered courses?


You 🫥:  i want to signup for a course


Bot 🤖: Kindly request for the course you want to register in this format:
(e.g.) My name is Promise, I am 30 years old, B-Tech Computer Science.
This is my phone number: 07063083925.
I want to learn Excel.


KeyboardInterrupt: Interrupted by user

In [183]:
target_response

'greetings'